# EFP (Exchange for Physical) Analysis

Interactive analysis notebook for Gold and Silver EFP metrics.

## What is EFP?
**EFP (Exchange for Physical)** is the cost/premium of converting a COMEX futures position into physical metal (or vice versa).

## Key Metrics
| Metric | Description |
|--------|-------------|
| **Swap-Implied EFP** | Theoretical cost based on financing rates: `Spot × Swap Rate × (Days to FDD / 360)` |
| **Traded EFP** | Actual market EFP: `Closing Futures - Closing Spot` |
| **Premium/Discount** | Difference: `Traded EFP - Swap Implied EFP` |

## Risk Thresholds
| Metal | Level 1 (50%) | Level 2 (75%) | Level 3 (100%) |
|-------|---------------|---------------|----------------|
| Gold | $4 | $6 | $8 |
| Silver | 15c | 20c | 25c |

In [ ]:
# Import required libraries
import bql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Import EFP modules
from efp_analysis import (
    generate_daily_efp_report,
    generate_historical_efp_data,
    generate_comex_inventory_history,
    print_daily_report,
    fetch_spot_price,
    fetch_futures_price,
    fetch_swap_rate,
    fetch_first_delivery_date,
    fetch_comex_stocks,
    fetch_open_interest,
    quick_efp_check,
    GOLD_TICKERS,
    SILVER_TICKERS,
    GOLD_THRESHOLDS,
    SILVER_THRESHOLDS,
)

from efp_charts import (
    plot_efp_timeseries,
    plot_efp_comparison,
    plot_premium_discount,
    plot_premium_discount_histogram,
    plot_comex_coverage_ratio,
    plot_comex_stocks_vs_oi,
    plot_swap_rate_term_structure,
    plot_swap_rate_history,
    plot_efp_dashboard,
    plot_all_efp_charts,
)

# Set display options
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 200)

# Initialize BQL
bq = bql.Service()

print("Libraries loaded successfully!")
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

---
## 1. Daily EFP Report

Generate current EFP metrics for Gold and Silver.

In [ ]:
# Generate daily EFP report
print("Generating Daily EFP Report...")
report = generate_daily_efp_report()

# Print formatted report
print_daily_report(report)

In [ ]:
# Create summary DataFrame
summary_data = []

for metal in ['gold', 'silver']:
    data = report[metal]
    if 'error' not in data:
        summary_data.append({
            'Metal': metal.upper(),
            'Spot': data['spot_price'],
            'Futures': data['futures_price'],
            'Days to FDD': data['days_to_fdd'],
            'Swap Rate': f"{data['swap_rate']*100:.2f}%",
            'Swap Implied EFP': data['swap_implied_efp'],
            'Traded EFP': data['traded_efp'],
            'Premium/Discount': data['premium_discount'],
            'Coverage Ratio': f"{data['coverage_ratio']*100:.1f}%",
            'Risk Status': data['risk_status'],
        })

summary_df = pd.DataFrame(summary_data)
display(summary_df)

---
## 2. Swap Rate Term Structure

View current swap rates from 1M to 7M tenor.

In [ ]:
# Fetch swap rates for Gold and Silver
print("Fetching swap rate term structure...")

gold_swaps = {}
silver_swaps = {}

for tenor in range(1, 8):
    try:
        gold_swaps[tenor] = fetch_swap_rate('GOLD', tenor)
    except Exception as e:
        print(f"Gold {tenor}M: Error - {e}")
        
    try:
        silver_swaps[tenor] = fetch_swap_rate('SILVER', tenor)
    except Exception as e:
        print(f"Silver {tenor}M: Error - {e}")

# Display term structure
print("\nGOLD Swap Rates:")
for tenor, rate in gold_swaps.items():
    print(f"  {tenor}M: {rate*100:.3f}%")

print("\nSILVER Swap Rates:")
for tenor, rate in silver_swaps.items():
    print(f"  {tenor}M: {rate*100:.3f}%")

In [ ]:
# Plot swap rate term structure
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gold
if gold_swaps:
    tenors = list(gold_swaps.keys())
    rates = [gold_swaps[t] * 100 for t in tenors]
    axes[0].plot(tenors, rates, 'o-', color='#FFD700', linewidth=2, markersize=10)
    axes[0].fill_between(tenors, 0, rates, alpha=0.3, color='#FFD700')
    axes[0].set_title('Gold Swap Rate Term Structure', fontsize=12, fontweight='bold')
    axes[0].set_xlabel('Tenor (Months)')
    axes[0].set_ylabel('Swap Rate (%)')
    axes[0].set_xticks(tenors)
    axes[0].set_xticklabels([f'{t}M' for t in tenors])
    axes[0].grid(True, alpha=0.3)
    for t, r in zip(tenors, rates):
        axes[0].annotate(f'{r:.2f}%', (t, r), textcoords='offset points',
                         xytext=(0, 10), ha='center', fontsize=9)

# Silver
if silver_swaps:
    tenors = list(silver_swaps.keys())
    rates = [silver_swaps[t] * 100 for t in tenors]
    axes[1].plot(tenors, rates, 'o-', color='#C0C0C0', linewidth=2, markersize=10)
    axes[1].fill_between(tenors, 0, rates, alpha=0.3, color='#C0C0C0')
    axes[1].set_title('Silver Swap Rate Term Structure', fontsize=12, fontweight='bold')
    axes[1].set_xlabel('Tenor (Months)')
    axes[1].set_ylabel('Swap Rate (%)')
    axes[1].set_xticks(tenors)
    axes[1].set_xticklabels([f'{t}M' for t in tenors])
    axes[1].grid(True, alpha=0.3)
    for t, r in zip(tenors, rates):
        axes[1].annotate(f'{r:.2f}%', (t, r), textcoords='offset points',
                         xytext=(0, 10), ha='center', fontsize=9)

plt.tight_layout()
plt.show()

---
## 3. Historical EFP Analysis

Generate historical EFP data for trend analysis.

In [ ]:
# Set date range for historical analysis
end_date = datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')  # 1 year

print(f"Historical Analysis Period: {start_date} to {end_date}")

In [ ]:
# Generate historical EFP data for Gold
print("Fetching Gold historical EFP data...")
gold_efp = generate_historical_efp_data('GOLD', start_date, end_date)

print(f"\nGold EFP Data Shape: {gold_efp.shape}")
print("\nLatest Data:")
display(gold_efp.tail())

In [ ]:
# Generate historical EFP data for Silver
print("Fetching Silver historical EFP data...")
silver_efp = generate_historical_efp_data('SILVER', start_date, end_date)

print(f"\nSilver EFP Data Shape: {silver_efp.shape}")
print("\nLatest Data:")
display(silver_efp.tail())

In [ ]:
# Plot Gold EFP Time Series
print("Generating Gold EFP Time Series Chart...")
fig = plot_efp_timeseries(gold_efp, 'GOLD')
plt.show()

print("\nInterpretation:")
print("- Blue line: Actual traded EFP (Futures - Spot)")
print("- Green dashed: Theoretical swap-implied EFP")
print("- Red shading: Premium (Traded > Implied) - physical demand")
print("- Green shading: Discount (Traded < Implied) - futures richness")

In [ ]:
# Plot Silver EFP Time Series
print("Generating Silver EFP Time Series Chart...")
fig = plot_efp_timeseries(silver_efp, 'SILVER')
plt.show()

In [ ]:
# Side-by-side comparison
print("Generating Gold vs Silver EFP Comparison...")
fig = plot_efp_comparison(gold_efp, silver_efp)
plt.show()

---
## 4. Premium/Discount Analysis

Analyze the premium/discount over time to identify market stress.

In [ ]:
# Plot Gold Premium/Discount
print("Generating Gold Premium/Discount Chart...")
fig = plot_premium_discount(gold_efp, 'GOLD')
plt.show()

# Statistics
prem_disc = gold_efp['premium_discount']
print("\nGold Premium/Discount Statistics:")
print(f"  Mean: ${prem_disc.mean():.2f}")
print(f"  Std Dev: ${prem_disc.std():.2f}")
print(f"  Min: ${prem_disc.min():.2f}")
print(f"  Max: ${prem_disc.max():.2f}")
print(f"  Current: ${prem_disc.iloc[-1]:.2f}")

In [ ]:
# Plot Silver Premium/Discount
print("Generating Silver Premium/Discount Chart...")
fig = plot_premium_discount(silver_efp, 'SILVER')
plt.show()

# Statistics (in cents)
prem_disc = silver_efp['premium_discount'] * 100
print("\nSilver Premium/Discount Statistics (in cents):")
print(f"  Mean: {prem_disc.mean():.1f}c")
print(f"  Std Dev: {prem_disc.std():.1f}c")
print(f"  Min: {prem_disc.min():.1f}c")
print(f"  Max: {prem_disc.max():.1f}c")
print(f"  Current: {prem_disc.iloc[-1]:.1f}c")

In [ ]:
# Premium/Discount Distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gold histogram
gold_prem = gold_efp['premium_discount'].dropna()
n, bins, patches = axes[0].hist(gold_prem, bins=40, edgecolor='white', alpha=0.7)
for patch, bin_left in zip(patches, bins[:-1]):
    if bin_left >= 0:
        patch.set_facecolor('#d62728')
    else:
        patch.set_facecolor('#2ca02c')
axes[0].axvline(x=0, color='black', linewidth=2)
axes[0].axvline(x=gold_prem.mean(), color='blue', linewidth=2, linestyle='--', label=f'Mean: ${gold_prem.mean():.2f}')
axes[0].set_title('Gold Premium/Discount Distribution', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Premium/Discount ($)')
axes[0].set_ylabel('Frequency')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Silver histogram
silver_prem = silver_efp['premium_discount'].dropna() * 100
n, bins, patches = axes[1].hist(silver_prem, bins=40, edgecolor='white', alpha=0.7)
for patch, bin_left in zip(patches, bins[:-1]):
    if bin_left >= 0:
        patch.set_facecolor('#d62728')
    else:
        patch.set_facecolor('#2ca02c')
axes[1].axvline(x=0, color='black', linewidth=2)
axes[1].axvline(x=silver_prem.mean(), color='blue', linewidth=2, linestyle='--', label=f'Mean: {silver_prem.mean():.1f}c')
axes[1].set_title('Silver Premium/Discount Distribution', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Premium/Discount (cents)')
axes[1].set_ylabel('Frequency')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## 5. COMEX Inventory Analysis

Analyze COMEX warehouse stocks and coverage ratios.

In [ ]:
# Fetch Gold COMEX inventory history
print("Fetching Gold COMEX inventory data...")
gold_comex = generate_comex_inventory_history('GOLD', start_date, end_date)

print(f"\nGold COMEX Data Shape: {gold_comex.shape}")
print("\nLatest Data:")
display(gold_comex.tail())

In [ ]:
# Fetch Silver COMEX inventory history
print("Fetching Silver COMEX inventory data...")
silver_comex = generate_comex_inventory_history('SILVER', start_date, end_date)

print(f"\nSilver COMEX Data Shape: {silver_comex.shape}")
print("\nLatest Data:")
display(silver_comex.tail())

In [ ]:
# Plot Gold COMEX Coverage Ratio
print("Generating Gold COMEX Coverage Chart...")
fig = plot_comex_coverage_ratio(gold_comex, 'GOLD')
plt.show()

print("\nCoverage Ratio Interpretation:")
print("- Coverage Ratio = COMEX Stocks / Open Interest (in oz)")
print("- >100%: More than enough metal to cover all contracts")
print("- 50-100%: Moderate stress possible")
print("- <50%: High stress - potential delivery squeeze")
print("- <25%: Critical - major delivery concerns")

In [ ]:
# Plot Silver COMEX Coverage Ratio
print("Generating Silver COMEX Coverage Chart...")
fig = plot_comex_coverage_ratio(silver_comex, 'SILVER')
plt.show()

In [ ]:
# Stocks vs Open Interest comparison
print("Generating Stocks vs Open Interest Charts...")

fig = plot_comex_stocks_vs_oi(gold_comex, 'GOLD')
plt.show()

fig = plot_comex_stocks_vs_oi(silver_comex, 'SILVER')
plt.show()

---
## 6. EFP Dashboard

Comprehensive view of all EFP metrics in a single dashboard.

In [ ]:
# Generate Gold EFP Dashboard
print("Generating Gold EFP Dashboard...")
fig = plot_efp_dashboard(gold_efp, gold_comex, 'GOLD')
plt.show()

print("\nDashboard Panels:")
print("1. Top: EFP Time Series (Traded vs Swap-Implied)")
print("2. Middle Left: Premium/Discount bars")
print("3. Middle Right: Swap Rate history")
print("4. Bottom Left: COMEX Stocks")
print("5. Bottom Right: Coverage Ratio")

In [ ]:
# Generate Silver EFP Dashboard
print("Generating Silver EFP Dashboard...")
fig = plot_efp_dashboard(silver_efp, silver_comex, 'SILVER')
plt.show()

---
## 7. Risk Threshold Analysis

Identify periods when EFP exceeded risk thresholds.

In [ ]:
# Analyze threshold breaches for Gold
print("GOLD THRESHOLD ANALYSIS")
print("=" * 50)

gold_prem = gold_efp['premium_discount'].abs()

level_1_breaches = (gold_prem >= GOLD_THRESHOLDS['level_1']).sum()
level_2_breaches = (gold_prem >= GOLD_THRESHOLDS['level_2']).sum()
level_3_breaches = (gold_prem >= GOLD_THRESHOLDS['level_3']).sum()

total_days = len(gold_prem)

print(f"\nTotal Trading Days: {total_days}")
print(f"\nLevel 1 (>=${GOLD_THRESHOLDS['level_1']}): {level_1_breaches} days ({level_1_breaches/total_days*100:.1f}%)")
print(f"Level 2 (>=${GOLD_THRESHOLDS['level_2']}): {level_2_breaches} days ({level_2_breaches/total_days*100:.1f}%)")
print(f"Level 3 (>=${GOLD_THRESHOLDS['level_3']}): {level_3_breaches} days ({level_3_breaches/total_days*100:.1f}%)")

# Current status
current = gold_efp['premium_discount'].iloc[-1]
print(f"\nCurrent Premium/Discount: ${current:.2f}")
if abs(current) >= GOLD_THRESHOLDS['level_3']:
    print("Status: LEVEL 3 - 100% cover recommended")
elif abs(current) >= GOLD_THRESHOLDS['level_2']:
    print("Status: LEVEL 2 - 75% cover recommended")
elif abs(current) >= GOLD_THRESHOLDS['level_1']:
    print("Status: LEVEL 1 - 50% cover recommended")
else:
    print("Status: Within normal range")

In [ ]:
# Analyze threshold breaches for Silver
print("SILVER THRESHOLD ANALYSIS")
print("=" * 50)

silver_prem = silver_efp['premium_discount'].abs()

level_1_breaches = (silver_prem >= SILVER_THRESHOLDS['level_1']).sum()
level_2_breaches = (silver_prem >= SILVER_THRESHOLDS['level_2']).sum()
level_3_breaches = (silver_prem >= SILVER_THRESHOLDS['level_3']).sum()

total_days = len(silver_prem)

print(f"\nTotal Trading Days: {total_days}")
print(f"\nLevel 1 (>={SILVER_THRESHOLDS['level_1']*100:.0f}c): {level_1_breaches} days ({level_1_breaches/total_days*100:.1f}%)")
print(f"Level 2 (>={SILVER_THRESHOLDS['level_2']*100:.0f}c): {level_2_breaches} days ({level_2_breaches/total_days*100:.1f}%)")
print(f"Level 3 (>={SILVER_THRESHOLDS['level_3']*100:.0f}c): {level_3_breaches} days ({level_3_breaches/total_days*100:.1f}%)")

# Current status
current = silver_efp['premium_discount'].iloc[-1]
print(f"\nCurrent Premium/Discount: {current*100:.1f}c")
if abs(current) >= SILVER_THRESHOLDS['level_3']:
    print("Status: LEVEL 3 - 100% cover recommended")
elif abs(current) >= SILVER_THRESHOLDS['level_2']:
    print("Status: LEVEL 2 - 75% cover recommended")
elif abs(current) >= SILVER_THRESHOLDS['level_1']:
    print("Status: LEVEL 1 - 50% cover recommended")
else:
    print("Status: Within normal range")

---
## 8. Summary

Quick overview of current EFP status.

In [ ]:
# Generate summary dashboard
print("=" * 80)
print(f"{'EFP ANALYSIS SUMMARY':^80}")
print(f"{'Generated: ' + datetime.now().strftime('%Y-%m-%d %H:%M'):^80}")
print("=" * 80)

for metal in ['GOLD', 'SILVER']:
    data = report[metal.lower()]
    thresholds = GOLD_THRESHOLDS if metal == 'GOLD' else SILVER_THRESHOLDS
    
    print(f"\n{metal}")
    print("-" * 40)
    
    if 'error' not in data:
        prem_disc = data['premium_discount']
        unit = '$' if metal == 'GOLD' else 'c'
        multiplier = 1 if metal == 'GOLD' else 100
        
        print(f"  Spot:              ${data['spot_price']:,.2f}")
        print(f"  Futures:           ${data['futures_price']:,.2f}")
        print(f"  Swap Implied EFP:  {unit}{data['swap_implied_efp']*multiplier:.2f}")
        print(f"  Traded EFP:        {unit}{data['traded_efp']*multiplier:.2f}")
        print(f"  Premium/Discount:  {unit}{prem_disc*multiplier:.2f}")
        print(f"  Coverage Ratio:    {data['coverage_ratio']*100:.1f}%")
        print(f"  Risk Status:       {data['risk_status']}")
    else:
        print(f"  Error: {data['error']}")

print("\n" + "=" * 80)

---
## Notes

### Bloomberg Tickers Used
| Ticker | Description |
|--------|-------------|
| `XAU Curncy` | Gold spot price |
| `XAG Curncy` | Silver spot price |
| `GCA Comdty` | Gold active futures contract |
| `SIA Comdty` | Silver active futures contract |
| `XAUSR1M-7M Curncy` | Gold swap rates (1-7 month) |
| `XAGSR1M-7M Curncy` | Silver swap rates (1-7 month) |
| `COMXGOLD Index` | COMEX gold warehouse stocks |
| `COMXSILV Index` | COMEX silver warehouse stocks |

### Key Formulas
- **Swap Implied EFP** = Spot × Swap Rate × (Days to FDD / 360)
- **Traded EFP** = Closing Futures - Closing Spot
- **Premium/Discount** = Traded EFP - Swap Implied EFP
- **Coverage Ratio** = COMEX Stocks / (Open Interest × Contract Size)